In [396]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import glob
tf.logging.set_verbosity(tf.logging.ERROR)
import numpy as np
from tensorboardX import SummaryWriter

In [397]:
# from youtube-8m utils.py
def Dequantize(feat_vector, max_quantized_value=2, min_quantized_value=-2):
  """Dequantize the feature from the byte format to the float format.

  Args:
    feat_vector: the input 1-d vector.
    max_quantized_value: the maximum of the quantized value.
    min_quantized_value: the minimum of the quantized value.

  Returns:
    A float vector which has the same shape as feat_vector.
  """
  assert max_quantized_value > min_quantized_value
  quantized_range = max_quantized_value - min_quantized_value
  scalar = quantized_range / 255.0
  bias = (quantized_range / 512.0) + min_quantized_value
  return feat_vector * scalar + bias

def decode(feat_vector, feature_size):
    return tf.reshape(tf.cast(tf.decode_raw(feat_vector, 
                                            tf.uint8), 
                              tf.float32),
                      [-1, feature_size])

In [398]:
# filepath is path to tfrecord
# datatype is audio or video
# output_features and output_labels are empty lists or existing lists
def load_data(filepath, data_type, output_labels, output_features):
    if data_type == 'audio':
        context = {
            'labels': tf.VarLenFeature(dtype=tf.int64)
        }

        feature_list = {
            'audio_embedding': tf.FixedLenSequenceFeature([], dtype=tf.string)
        }
        feature_name = 'audio_embedding'
        feature_len = 128

    elif data_type == 'video':
        context = {
            'id': tf.FixedLenFeature([], dtype=tf.string),
            'labels': tf.VarLenFeature(dtype=tf.int64)
        }

        feature_list = {
            'rgb': tf.FixedLenSequenceFeature([], dtype=tf.string),
        }
        feature_name = 'rgb'
        feature_len = 128
        
        
    tf.reset_default_graph()    


    # Read TFRecord file
    reader = tf.TFRecordReader()
    filename_queue = tf.train.string_input_producer([filepath])


    # Extract features from serialized data

    _, serialized_example = reader.read(filename_queue)
    context, features = tf.io.parse_single_sequence_example(serialized_example,
                                                    context_features=context,
                                                    sequence_features=feature_list,
                                                    example_name=None,
                                                    name=None
    )
    labels = context['labels']
    label = labels.values[0]
    data = Dequantize(decode(features[feature_name], feature_len))

    # Many tf.train functions use tf.train.QueueRunner,
    # so we need to start it before we read
    
    
    with tf.Session() as sess:
        sess.run(tf.local_variables_initializer())
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        # f = codecs.open(outpath, "w", encoding='utf-8')
        try:
            counter = 0
            recordlist = []

            num_in_file = sum(1 for _ in tf.python_io.tf_record_iterator(filepath))

            for i in range(num_in_file):
                d, l = sess.run([data, label])
                output_labels.append(l)
                output_features.append(d)
        except tf.errors.OutOfRangeError:
            print('Finished extracting from tfrecord data.')
        finally:
            coord.request_stop()
            coord.join(threads)


    
    return output_labels, output_features

In [399]:
audio_output_labels = []
audio_output_features = []
audio_path = "audio_1556878148.412379.tfrecord"
audio_output_labels, audio_output_features = load_data(audio_path,
                                           'audio', audio_output_labels, audio_output_features)

video_output_labels = []
video_output_features = []
video_path = "video_1556875795.817071.tfrecord"
video_output_labels, video_output_features = load_data(video_path,
                                           'video', video_output_labels, video_output_features)

In [400]:
audio_x = np.array(audio_output_features)
audio_y = np.array(audio_output_labels)
video_x = np.array(video_output_features)

video_y = np.array(video_output_labels)
print('audio_x shape: ', audio_x.shape)
print('audio_y shape: ', audio_y.shape)
print('video_x shape: ', video_x.shape)
print('video_y shape: ', video_y.shape)


audio_x shape:  (2243, 10, 128)
audio_y shape:  (2243,)
video_x shape:  (2243, 10, 128)
video_y shape:  (2243,)


In [ ]:
print(np.sort(audio_y))
print(np.sort(video_y))

In [ ]:
# --- At this point the desired data should be loaded --- 

In [411]:
def build_batch(dataset_x, dataset_y, batch_size, labels=None):
    if labels is None:
        indices = list(np.random.randint(0, len(dataset_x), size=batch_size))
    else:
        indices=[]
        for i in labels:
            #print(i)
            indices.append(np.where(dataset_y == i)[0][0])
    # Recover what the entries for the batch are
    batch_x = np.array([dataset_x[i] for i in indices])
    batch_y = np.array([dataset_y[i] for i in indices])
    
    return batch_x, batch_y, indices

In [412]:
def contrastive_loss(embedded_batch_a, embedded_batch_b, a_labels, b_labels, margin):
    y = tf.cast(tf.equal(a_labels, b_labels), tf.float32)
    dist = tf.norm(embedded_batch_a - embedded_batch_b, axis=1)
    print(dist.shape)
    loss = (y) * .5 * tf.square(dist) + (1-y) * .5 * tf.square(tf.maximum(0., margin - dist))
    return y, tf.reduce_sum(loss)

In [413]:
def build_graph(input_shape, labels_shape, 
                embed_size=128, learning_rate=0.001, 
                l1_reg=0.001, l2_reg=0.001, margin=50):
    audio_inputs = tf.placeholder(shape=[None, input_shape[0], input_shape[1]], name="audio_inputs", dtype=tf.float32)
    video_inputs = tf.placeholder(shape=[None, input_shape[0], input_shape[1]], name="video_inputs", dtype=tf.float32)
    audio_labels = tf.placeholder(shape=[None], name="audio_labels", dtype=tf.float32)
    video_labels = tf.placeholder(shape=[None], name="video_labels", dtype=tf.float32)

    flattened_audio = tf.layers.flatten(audio_inputs)
    flattened_video = tf.layers.flatten(video_inputs)
    flattened_audio_labels = tf.layers.flatten(audio_labels)
    flattened_video_labels = tf.layers.flatten(video_labels)

    
    #subnetwork = build_fc_net(inputs, embed_size, np.prod(video_input_shape), l1_reg, l2_reg)
    weights = tf.trainable_variables()
    if l1_reg != 0:
        reg_1 = tf.contrib.layers.l1_regularizer(scale=l1_reg)
    else:
        reg_1 = 0
    if l2_reg != 0:
        reg_2 = tf.contrib.layers.l2_regularizer(scale=l2_reg)
    else:
        reg_2 = 0
    audio_embed = build_fc_net(flattened_audio, embed_size, l1_reg, l2_reg)
    video_embed = build_fc_net(flattened_video, embed_size, l1_reg, l2_reg, reuse=True)
    weights = tf.trainable_variables()

    #mse = tf.losses.mean_squared_error(audio_embed, video_embed) # only use loss if labels don't match?
    matches, error = contrastive_loss(audio_embed, video_embed, flattened_audio_labels, flattened_video_labels, margin)
    reg_penalty = tf.contrib.layers.apply_regularization(reg_1, weights) + tf.contrib.layers.apply_regularization(reg_2, weights)
    loss = error + reg_penalty
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)
    return (audio_inputs, audio_labels,
            video_inputs, video_labels,
            loss, optimizer, matches,
            audio_embed, video_embed)

In [414]:
#audio input is a tf placeholder for the input audio features
#video labels is a tf placeholder for the input video features
#encode size is the desired size of the encoded vector (should be the same size as the video features)
#l1 and l2 reg are the amount of weight to put on l1 and l2 regularizers for the loss

def build_fc_net(input_data, embed_size, l1_reg=0.001, l2_reg=0.001, reuse=False):
    net = build_fc_layers(input_data, embed_size, reuse)
    return net

In [415]:
# input layer should be the flattened inputs
def build_fc_layers(input_layer, output_size, reuse=False):
    with tf.name_scope("model"):
        with tf.variable_scope("dense0", reuse=reuse) as scope:
            h1 = tf.layers.dense(inputs=input_layer, units=1024, activation=tf.nn.tanh)
            d1 = tf.layers.dropout(inputs=h1, rate=.3)
        with tf.variable_scope("dense1", reuse=reuse) as scope:
            h2 = tf.layers.dense(inputs=d1, units=512, activation=tf.nn.tanh)
            d2 = tf.layers.dropout(inputs=h2, rate=.3)            
        with tf.variable_scope("dense2", reuse=reuse) as scope:
            h3 = tf.layers.dense(inputs=d2, units=256, activation=tf.nn.tanh)
            d3 = tf.layers.dropout(inputs=h3, rate=.3)
        with tf.variable_scope("dense3", reuse=reuse) as scope:
            raw_encode = tf.layers.dense(inputs=d3, units=output_size, activation=tf.nn.tanh)

    return raw_encode

In [416]:
def train(sess, audio_inputs, audio_labels, 
          video_inputs, video_labels, 
          audio_x, audio_y,
          video_x, video_y,
          loss, optimizer, 
          audio_embed, video_embed, 
          batch_size, cur_epoch, num_iters=1000):

        for i in range(num_iters):
            if i % 2 == 0:
                audio_batch_input, audio_batch_label, _ = build_batch(audio_x, audio_y, batch_size)
                video_batch_input, video_batch_label, _ = build_batch(video_x, video_y, batch_size)
            else:
                audio_batch_input, audio_batch_label, indices = build_batch(audio_x, audio_y, batch_size)
                video_batch_input, video_batch_label, _ = build_batch(video_x, video_y, batch_size, audio_batch_label)                
            _, loss_val,ys = sess.run([optimizer, loss, matches], feed_dict={audio_inputs: audio_batch_input,
                                                             video_inputs: video_batch_input,
                                                             audio_labels: audio_batch_label,
                                                             video_labels: video_batch_label})
            if i % 100 == 0:
                train_summary_writer.add_scalar('train_loss', loss_val, cur_epoch * num_iters + i)
                #print(ys)
                #print(loss_val)

In [417]:
def k_nearest_neighbors(embedding, search_space, k):
    embedding_stacked = np.repeat(embedding[np.newaxis,:], len(search_space), axis=0)
    similarity = np.linalg.norm(embedding_stacked - search_space, axis=1)
    #print(similarity.shape)
    indices = np.argsort(similarity)[:k]
    #print(indices)
    return indices

def validate(sess, audio_inputs, audio_labels, 
             video_inputs, video_labels, 
             audio_x, audio_y,
             video_x, video_y,
             loss, optimizer, 
             audio_embed, video_embed, 
             cur_epoch, batch_size=100, k=5):

    audio_batch_input, audio_batch_label, indices = build_batch(audio_x, audio_y, batch_size)
    #print(audio_batch_label)
    video_batch_input, video_batch_label, _ = build_batch(video_x, video_y, batch_size, audio_batch_label)
    #print(video_batch_label)
    audio_embeddings = sess.run(audio_embed, feed_dict={audio_inputs: audio_batch_input,
                                                          audio_labels: audio_batch_label})
    video_embeddings = sess.run(video_embed, feed_dict={video_inputs: video_batch_input,
                                                          video_labels: video_batch_label})
    matches = 0
    for i in range(len(audio_embeddings)):
        a = audio_embeddings[i]
        if i%10 == 0:
            print(a)
        a_y = audio_batch_label[i]
        ids = k_nearest_neighbors(a, video_embeddings, k)
        #print(a_y)
        #print(ids)
        if a_y in video_batch_label[ids]:
            matches += 1
    train_summary_writer.add_scalar('found_matches', matches/batch_size, cur_epoch)

In [418]:
train_log_dir = 'logs/tensorboard/train/log4'
train_summary_writer = SummaryWriter(train_log_dir)


num_epochs = 50
batch = 50
m=5

tf.reset_default_graph()

(audio_inputs, audio_labels, 
 video_inputs, video_labels, 
 loss, optimizer, matches,
 audio_embed, video_embed) = build_graph(audio_x[0].shape, audio_y.shape[0], margin=m)

sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(num_epochs):
    validate(sess, audio_inputs, audio_labels,
          video_inputs, video_labels,
          audio_x, audio_y, 
          video_x, video_y, 
          loss, optimizer, 
          audio_embed, video_embed, epoch)

    train(sess, audio_inputs, audio_labels,
          video_inputs, video_labels,
          audio_x, audio_y, 
          video_x, video_y, 
          loss, optimizer, 
          audio_embed, video_embed, batch, epoch)



(?,)
[-0.34381562  0.25493842  0.04259005 -0.24351642 -0.03280356  0.17262056
  0.16193756  0.17236963 -0.721403    0.2904201  -0.6236148   0.1485264
 -0.45117962  0.04514455 -0.6001738   0.45453867 -0.20184839 -0.8604697
  0.6643472   0.11627258  0.05766319 -0.29702997 -0.3232111   0.79607606
  0.47949398 -0.05132649 -0.31866643  0.21243022 -0.2884724   0.43780065
 -0.6228613   0.67874557 -0.7104932  -0.05327629  0.3165896   0.7329378
 -0.2518558   0.2513222   0.01759174  0.77812946  0.5082891  -0.31406164
 -0.08141109  0.23584655 -0.41402176 -0.4314852   0.7082617   0.31912684
  0.61804163  0.18182124 -0.7009201  -0.34518296  0.4229646  -0.03416296
  0.62340117  0.5586779   0.24903125 -0.4087584   0.73383486  0.5264662
  0.46896136  0.01595339 -0.5420681  -0.42498726  0.2162628   0.16577807
  0.31876832  0.11874498  0.17158616 -0.02732659 -0.5370427  -0.42788523
  0.2743457   0.00842978  0.5663247  -0.12267174 -0.32279447  0.38702703
  0.16686507  0.59345984 -0.5885924   0.15516868 -

[ 0.650663    0.30384836  0.20645125  0.03420113 -0.3999148   0.408195
  0.01664033  0.5409006   0.49774924 -0.02500606  0.48643598 -0.6988809
  0.2506254  -0.15643269  0.2874852   0.31080264  0.11273459 -0.57391816
 -0.17672516 -0.15092872 -0.75409734 -0.08612478  0.27051637 -0.1618268
  0.38956854 -0.76289403 -0.00941955 -0.45653677 -0.15151379 -0.22863643
  0.25897077  0.91218674 -0.48117366 -0.34396935 -0.48115584  0.23636664
 -0.01193315  0.37487856  0.01084759 -0.3558756   0.26295128  0.4677412
  0.28899375 -0.56367993 -0.17002814 -0.11545264  0.18228228 -0.03051442
 -0.1400108   0.36550215  0.38421226 -0.03455869  0.2956972  -0.71729445
 -0.30184028 -0.44396192 -0.02164929  0.24114808 -0.4725485  -0.15434183
 -0.7408762  -0.11197346 -0.72897035 -0.04462679  0.57658565 -0.6767628
  0.35096538  0.39372244  0.28110805 -0.28155038  0.58564144 -0.14961793
 -0.65278494 -0.33018228  0.41559553  0.18596736  0.76472664  0.6771154
 -0.11205054 -0.19026348  0.02745303  0.54075146 -0.027561

[ 0.34841725 -0.67679226 -0.6861706   0.37383392 -0.15783623  0.15052354
 -0.77674764  0.5217706  -0.04875059  0.5115785   0.24449006  0.19700652
  0.6177368   0.28413612 -0.544057   -0.32928732 -0.30330312 -0.21014448
  0.5119862  -0.1463608  -0.5134934   0.26234424  0.1113335  -0.01927049
  0.09233166 -0.49559426  0.12341759 -0.01902669 -0.33425364 -0.46219397
  0.42156744  0.43081012 -0.18962198 -0.73709166 -0.28680322  0.23535064
  0.37948567  0.13228957  0.00184433  0.6021266  -0.50417536  0.45990524
  0.589606    0.2291063   0.5411436  -0.59424305 -0.49214262 -0.06662382
 -0.10880215 -0.6282612  -0.01622482  0.2408812   0.69137174 -0.6188061
 -0.8343591  -0.6264844  -0.44761792  0.13249609 -0.18580179  0.21923189
  0.04098412 -0.23005512 -0.42936656 -0.59793174 -0.0035523  -0.25131884
  0.4965912   0.65087044  0.6175144  -0.19796672 -0.2763526   0.39966813
 -0.37175006 -0.50703835 -0.02919998 -0.19226943  0.25752184  0.58866584
 -0.59607023  0.13323225 -0.7837678   0.3544633  -0.

[-0.6057305  -0.2534514  -0.10532176  0.72605145  0.6181503  -0.2850089
 -0.11353867 -0.5383264  -0.2867931  -0.29594246 -0.18543805  0.6488627
 -0.5468184   0.24513878 -0.5027807  -0.72501093 -0.22454737 -0.5985895
  0.08299416 -0.63261867  0.56847763  0.71263504 -0.6341308   0.3175114
 -0.39727172  0.10542376 -0.40323296  0.4789065  -0.09066442  0.07218963
 -0.366637   -0.6387447   0.5722556   0.41822124  0.10381041 -0.20696686
  0.5228995  -0.06590207 -0.44558308  0.6588882  -0.2205849  -0.44982275
 -0.03662559  0.54943264  0.01145628 -0.02931385 -0.10153899  0.09814187
 -0.04515443 -0.12880877 -0.571522   -0.2490059  -0.03438217  0.4873262
 -0.03449549  0.3985766  -0.42986873 -0.5844594   0.6563684   0.1713074
  0.6236744   0.07371467  0.5114248  -0.21714377 -0.7571292   0.6068215
 -0.2760484  -0.37135288  0.3096274   0.6645985  -0.5679442   0.13631116
  0.8567919   0.6568232  -0.5110471  -0.12186255 -0.8539492  -0.1860792
  0.41109967 -0.6662107   0.00240749 -0.47296095  0.1636293

[ 0.44743407  0.61155105  0.7817571  -0.34282163  0.7671057   0.27491596
  0.30726883 -0.5359091  -0.63160205 -0.7796334  -0.42787817 -0.5071384
 -0.7231935   0.7164753   0.57369864  0.13959305 -0.7469567   0.72357005
 -0.855312    0.7701489   0.2059565   0.7198365   0.7125115   0.7399851
 -0.7856679   0.6092402   0.7301103  -0.6148758   0.8178876   0.75734395
 -0.29518008 -0.5841992   0.8907602   0.860296   -0.53823566  0.90622014
 -0.41542304 -0.7562446  -0.7896992  -0.1955979  -0.46489066 -0.50640625
 -0.83285964  0.03863014  0.58163214  0.53215045  0.2122089  -0.86499417
 -0.78199697  0.7839594  -0.6978462   0.33873522 -0.5197406   0.6139156
  0.45249593  0.7174153   0.8758144  -0.8687137  -0.40028048 -0.2768823
 -0.06723728  0.5269568   0.5736145   0.46227112  0.6219579   0.71693444
 -0.05277795 -0.8206498  -0.7279793   0.7917278   0.696664    0.10885017
  0.7962878  -0.21163037 -0.54361546 -0.50780463 -0.24352579 -0.4997554
  0.1214435  -0.49601826  0.75917906 -0.1721184   0.7609

...

[ 0.9997229   0.999709   -0.9998578   0.9997435  -0.999841    0.992071
  0.9998423  -0.99980676  0.9998331   0.99970824  0.9996235  -0.9999993
  0.99983734  0.9997999   0.9995867  -0.99754107 -0.9997617   0.99988765
 -0.9999073  -0.9997972  -0.9997848   0.09355189  0.9997403   0.9998777
 -0.99999976  0.99985504 -0.9998983  -0.99981654  0.9998429  -0.9994396
  0.999831   -0.9349349   0.9998201  -1.          0.99985385  0.99972254
  0.9998535   0.99978137 -0.9997819  -0.9998047   0.99975693 -0.9997783
  0.9997816  -0.99980646  0.99977636  0.9998931  -0.9998041  -0.9997108
 -0.99992615  0.9999982  -0.99936306  0.99953854  0.9996652   0.9999922
  0.9999001  -0.9998495  -0.99977094  0.9998911  -0.999884    0.9997929
 -0.99999934  0.99978715 -0.99993616  0.9998783   0.70992154  0.99963945
 -0.9999999  -0.9997162  -0.9998484   0.99981946 -0.99967724 -0.9998445
 -0.9997891   0.99999636 -0.9998403   0.9998426  -0.9997036  -0.9998601
 -0.99991584 -0.17451447 -0.99979293 -0.99988234 -0.99982196  

[ 0.9991983   0.99869657 -0.9992613   0.9987529  -0.9991884   0.9999657
  0.99929786 -0.999095    0.99932814  0.9996693   0.99855417 -0.9999965
  0.99916583  0.99903595  0.9987603  -0.99637616 -0.99908316  0.9995009
 -0.99959916 -0.99915147 -0.99920505 -0.97951627  0.99912226  0.99947053
 -0.99998367  0.99935573 -0.9995678  -0.9992759   0.99932414 -0.9981722
  0.99930084 -0.9908855   0.99927056 -1.          0.9992818   0.9990025
  0.99898356  0.9991126  -0.9991301  -0.9991509   0.9986757  -0.9991386
  0.9987854  -0.9991462   0.9985103   0.9994338  -0.9991726  -0.9990127
 -0.9995769   0.9996385  -0.9982083   0.99836767  0.9986729   0.9955829
  0.99945694 -0.9991798  -0.99880356  0.99936813 -0.99952173  0.9991123
 -0.9999991   0.998971   -0.9995473  -0.9641622   0.83302     0.99866134
 -0.9999989  -0.9990559  -0.9993601   0.9990259  -0.9985961  -0.9993648
 -0.9990541   0.99999344 -0.99935377  0.9993644  -0.99869716 -0.9993632
 -0.99960124  0.99968356 -0.99901974 -0.999493   -0.9991227   

[ 0.9996121   0.99823016 -0.9993021   0.99945664 -0.99876666  0.9999906
  0.99855566 -0.99820364  0.99843985  0.99954486  0.9992337  -1.
  0.99840444  0.9983922   0.9994443  -0.99995404 -0.9987315   0.9984946
 -0.9988197  -0.99852836 -0.9986421  -0.9984672   0.99899745  0.9993015
 -0.9987753   0.99912053 -0.99867195 -0.99869174  0.9986393  -0.99864936
  0.9985432   0.18146813  0.99907124 -1.          0.9999798   0.9985328
  0.9982963   0.9989922  -0.9988761  -0.9985489   0.9988279  -0.9984055
  0.9977914  -0.99875     0.99796784  0.9994737  -0.9997913  -0.9980583
 -0.9987435   0.99999565 -0.9995334   0.9981248   0.9982379   0.9982742
  0.99864167 -0.99859744 -0.9986251   0.9986994  -0.9985055   0.99874777
 -0.9999579   0.9986517  -0.9989164   0.76720244  0.99962485  0.99922764
 -0.9999863  -0.9982646  -0.9984872   0.99848133 -0.9989479  -0.99872255
 -0.99845314  0.9999862  -0.9987415   0.9983606  -0.9982589  -0.99861515
 -0.9986845   0.9999105  -0.9987759  -0.99858403 -0.99844575  0.99

[ 0.99894154  0.99986506 -0.9982996   0.9982256  -0.9983497   0.04891155
  0.9994286  -0.9997768   0.9991325   0.9996316   0.9983628   0.9996075
  0.9989628   0.9989871   0.99879855 -0.9995516  -0.9987582   0.998793
 -0.99882597 -0.99883085 -0.9986712   0.9908442   0.99854606  0.99907327
 -0.99969494  0.9987663  -0.99886936 -0.9988331   0.9988383  -0.99882656
  0.9989622  -0.9966768   0.99871457 -1.          0.99792564  0.9989171
  0.9982621   0.99855405 -0.998649   -0.9993075   0.9985405  -0.9988637
  0.998409   -0.99884677  0.9990316   0.9989863  -0.9981034  -0.9999999
 -0.99998903 -0.940586   -0.99997014  0.9994398   0.9991416   0.99946725
  0.99945045 -0.99866587 -0.99838156  0.9983476  -0.99884856  0.9987829
  0.9970155   0.9992664  -0.9982705   0.9183001  -1.          0.9982306
 -0.9999303  -0.99947804 -0.99866307  0.9987546  -0.99819535 -0.998743
 -0.9987747   0.99999577 -0.99891156  0.99905086 -0.998886   -0.99895716
 -0.9987177  -0.9998396  -0.99889624 -0.9989983  -0.9989261  

[ 0.99992245  0.99994713 -0.99991626  0.9999006  -0.99992794 -0.9997024
  0.9999496  -0.9999597   0.99993855  0.9999417   0.99988824  0.99804664
  0.9999301   0.999922    0.9999429  -0.99990517 -0.99991214  0.9998993
 -0.99992317 -0.99992466 -0.9999155  -0.99999154  0.9999008   0.99992806
  0.9997503   0.9999049  -0.99992776 -0.9999196   0.9999175  -0.9999052
  0.99992317  0.9980277   0.9999014  -0.99999994  0.99998283  0.9999377
  0.99993414  0.9999226  -0.99989456 -0.99993235  0.99990916 -0.9999313
  0.99991846 -0.9999108   0.9999498   0.99991405 -0.99990296 -0.9999997
 -0.99993837  0.91447914 -0.99993795  0.99994576  0.9999485   0.9999924
  0.9999361  -0.9999388  -0.9999293   0.999912   -0.99992514  0.9999099
  0.9993005   0.9999506  -0.999907    0.2708299  -0.9999999   0.99988806
 -0.99993813 -0.9999465  -0.9999274   0.9999128  -0.9999234  -0.99992335
 -0.99991137  0.9999721  -0.9999216   0.9999404  -0.99991876 -0.99987346
 -0.9999033  -0.9720229  -0.9999162  -0.99993294 -0.9999295

In [ ]:
validate(sess, audio_inputs, audio_labels,
      video_inputs, video_labels,
      audio_x, audio_y, 
      video_x, video_y, 
      loss, optimizer, 
      audio_embed, video_embed, epoch)